# <span style="color:#0b486b">FIT3162 MCS13 Code</span>


---

## <span style="color:#0b486b">Imports</span>

Library imports required for this file. 
- `os` - OS related functions (getting/setting current directory or filepath(s))
- `zipfile` - to unzip files
- `PIL.Image` -  Image related functions
- `sklearn.model_selection.train_test_split` - A function to split up datasets
- `tensorflow` - General Deep Learning purposes
  - `keras`
    - `layers, models, activations` - for Deep Learning Model creation
- `matplotlib.pyplot` - for plotting graphs (if needed)
- `numpy` - for array types (needed for tensorflow functions)
- `BaseModel` - custom CNN Model class
- `ResNetModel` - custom CNN Model class, with ResNet blocks, Batch Normalisation and Dropout Layers
- `SkipConnModel` - custom CNN Model class, with ResNetModel features and Skip Connection. Some Skip Connection with Conv2D layers too.
- `constants` - Constant values used throughout the file

In [ ]:
""" Code Header
This file contains code thats trains and tests Models for Image Classification.
Various Model architectures are used and tried. 

In order to make use of the dataset files, ensure that the download.py file under the RealWorldOccludedFaces-main directory is run already.

Note: 
This jupyter file was obtained and altered from the Google Colab file from the MCS13 private Google Drive
The file was created on 08/03/2024 and contained code cells for: 
- Image loading
- Image resizing
- Creation of some simple models (obtained and altered from GeminiAI and Microsoft Copilot)
- Model training

@author MCS13
@version 1.3.0
@since 29/03/2024
@updated 05/04/2024
"""

# ============================================================================================================= #

# Imports 
import os
import zipfile
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, activations
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
import numpy as np

from typing import List, Tuple
from numpy.typing import NDArray

from BaseModel import BaseModel
from ResNetModel import ResNetModel
from SkipConnModel import SkipConnModel
from constants import *

plt.style.use('ggplot')
%matplotlib inline


---

## <span style="color:#0b486b">Loading Datasets</span>

***[RUN ONCE]*** Unzip and extract all files.

In [ ]:
def unzip_file(filepath: str):
    """Unzips a zip file
    This code unzips a file and saves the content in the current directory.
    This means the content will be under the same directory as the original zip file.

    @since 1.0.0

    @prerequisite: The input filepath must end with ".zip"
    @raise ValueError: When the input filepath does not end with ".zip"

    @param filepath(str): The file path to unzip. The file path must lead to a zip file
    """
    # Pre-requisite check
    if filepath[-4:] != ".zip":
        raise ValueError(
            f"The input filepath must end with '.zip'. Expected filepath = '<FileName>.zip', got filepath = '{filepath}'."
        )

    # Unzip the file
    zip_obj = zipfile.ZipFile(file=filepath, mode="r")
    zip_obj.extractall("./")
    zip_obj.close()


"""RUN ONCE: Unzipping the dataset zip file """
# dataset_zip_filepath = f"{DATASET_NAME_MAIN}.zip"
# unzip_file(dataset_zip_filepath)

Here, we start with the neutral faces dataset.

In [ ]:
print(
    f'A few samples of people in the dataset: {os.listdir("RealWorldOccludedFaces-main/images/neutral")[:5]}'
)
print(
    f'Number of unique ids: {len(os.listdir("RealWorldOccludedFaces-main/images/neutral"))}'
)
print(
    f'Some important constants:\nRESIZED_SHAPES: {RESIZED_SHAPES}\nWIDTH: {WIDTH}\nHEIGHT: {HEIGHT}\n'
)

***[RUN ONCE]*** Resizes all images to a fixed shape of (64, 64)

In [ ]:
# Main file paths
main_path = f"{DATASET_NAME_MAIN}/{NEUTRAL_DIR}"
main_dirs = os.listdir(main_path)
num_classes = len(main_dirs)

def resize_neutral_images(resized_shape: Tuple[int, int]):
    """Resizes all neutral images
    The function takes all images in RealWorldOccludedFaces-main/images/neutral
      and resizes them to 224x224 (determined by RESIZED_SHAPE constant)

    The resized shapes are then saved in RealWorldOccludedFaces-resized/images/neutral

    @param resized_shape(Tuple[int, int]): The shape to resize the shape to.
    """
    # Creates the directory paths
    resized_path = f"{DATASET_NAME_RESIZED}_{resized_shape[0]}/{NEUTRAL_DIR}"

    for path in [os.path.join(resized_path, f) for f in os.listdir(main_path)]:
        os.makedirs(f"{path}")
    
    # File paths of the neutral images
    paths = [
        os.path.join(resized_path, f)
        for f in main_dirs
    ]

    # Resizing and saving all images
    for name in os.listdir(main_path):
        for img_name in os.listdir(f"{main_path}/{name}"):
            Image.open(os.path.join(f"{main_path}/{name}/", img_name)).resize(
                resized_shape
            ).save(os.path.join(f"{resized_path}/{name}/", img_name))


# Creating the directory paths before resizing the images
# Needed before saving files
"""RUN ONCE: Resizing image dataset"""
# resize_neutral_images(RESIZED_SHAPES[3])

Function to load the images into numpy arrays for model input.

In [ ]:
def get_neutral_image_data(
    resized_shape: Tuple[int, int] = (64, 64)
) -> Tuple[NDArray[List[List[int]]], NDArray[NDArray[List[List[int]]]]]:
    """Generates the dataset values (as ndarrays)
    Produces the dataset values (x-values) and
        the one-hot vector encoding of their labels (y-values)

    The values are obtained from the neutral images

    @rtype: Tuple[ndarray[List[List[int]]], ndarray[ndarray[List[List[uint8]]]]]
    @return: x-values, y-values
    """
    resized_path = f"{DATASET_NAME_RESIZED}_{resized_shape[0]}/{NEUTRAL_DIR}"

    # File paths of the neutral images
    paths = [os.path.join(resized_path, f) for f in main_dirs]

    faces = []
    ids = []

    for unique_id, path in enumerate(paths):
        for img in [os.path.join(path, f) for f in os.listdir(path)]:
            image = np.array(Image.open(img), "uint8")

            # Translate the image laterally and vertically by a random amount.
            translated_image_lateral = np.roll(image, np.random.randint(-5, 5), axis=1)
            translated_image_vertical = np.roll(image, np.random.randint(-5, 5), axis=0)

            # Add the original, laterally translated, and vertically translated images to the list.
            faces.append(image)
            faces.append(translated_image_lateral)
            faces.append(translated_image_vertical)

            # Add the corresponding labels to the list.
            id = [int(bit == unique_id) for bit in range(len(os.listdir(main_path)))]
            ids.append(id)
            ids.append(id)
            ids.append(id)

    # Return the dataset values
    return np.array(faces), np.array(ids)

In [ ]:
# Takes a while...
faces, ids = get_neutral_image_data(RESIZED_SHAPES[0])

In [ ]:
print(f"Number of face images: {len(faces)}")  # size of neutral faces dataset
print(f"Number of ids (must match above): {len(ids)}")  # size of the array of face ids

Splitting the dataset into training, validation and testing.

In [ ]:
# Splitting the dataset into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(
    faces, ids, test_size=0.2, random_state=42
)

# Splitting the training dataset into training and validation datasets
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=42
)


--- 

## <span style="color:#0b486b">Modelling time!</span>

In [ ]:
model = BaseModel(
    name="Basic Model",
    input_width=RESIZED_SHAPES[0][0],
    input_height=RESIZED_SHAPES[0][1],
    depth=DEPTH,
    num_classes=num_classes,
    activation_func=RELU,
    optimiser=ADAM_OPT,
)

model.build_cnn()
model.summary()

In [ ]:
# Commented out since we don't use this model anymore
# model.fit(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)

Train the model

In [ ]:
# Commented out since we don't use this model anymore
# evaluation_results = model.compute_accuracy(x_test, y_test)


---

## <span style="color:#0b486b">ResNet Upgrade!</span>

Now, we upgrade the model by implementing **ResNet blocks**, **Dropout layers** and **Batch Normalisation**

In [ ]:
resnet_model = ResNetModel(
    name="ResNet Model",
    input_width=RESIZED_SHAPES[0][0],
    input_height=RESIZED_SHAPES[0][1],
    depth=DEPTH,
    num_classes=num_classes,
    activation_func=RELU,
    optimiser=ADAM_OPT,
)

resnet_model.build_cnn()
resnet_model.summary()

In [ ]:
# Commented out since we don't use this model anymore
# resnet_model.fit(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)

In [ ]:
# Commented out since we don't use this model anymore
# evaluation_results = resnet_model.compute_accuracy(x_test=x_test, y_test=y_test)


---

## <span style="color:#0b486b">SkipConnection Upgrade!</span>

Now, we upgrade the model by implementing **ResNet blocks**, **Dropout layers** and **Batch Normalisation**

In [ ]:
skip_conn_model = SkipConnModel(
    name="ResNet Model",
    input_width=WIDTH,
    input_height=HEIGHT,
    depth=DEPTH,
    num_classes=num_classes,
    num_epochs = 30,
    activation_func=RELU,
    optimiser=ADAM_OPT,
)

skip_conn_model.build_cnn()
skip_conn_model.summary()

In [ ]:
# Stops early based on the validation loss
# When running with GPU, could use higher patience level (cuz can afford to compute more hahah)
# min_delta parameter is used to determine how much leeway to give for change of validation loss
#       default is 0
early_stopping = EarlyStopping(patience=5, monitor="val_loss", mode="min", min_delta=0)

# Saves the best model based on validation loss
val_loss_checkpoint = ModelCheckpoint(
    os.path.join("./ckpts", "best_val"),
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    verbose=1,
)

# Saves the best model based on validation accuracy
val_acc_checkpoint = ModelCheckpoint(
    os.path.join("./ckpts", "best_acc"),
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    verbose=1,
)

skip_conn_model.fit(
    x_train=x_train,
    y_train=y_train,
    x_val=x_val,
    y_val=y_val,
    callbacks=[early_stopping, val_loss_checkpoint, val_acc_checkpoint],
)

In [ ]:
evaluation_results = skip_conn_model.compute_accuracy(x_test=x_test, y_test=y_test)

#### **Version Overview**

1.0.0
- Copied file from jupyter notebook in MCS13 private Google Drive
- Imported new BaseModel and constants files
- Did basic training using BaseModel 

1.1.0
- Imported ResNetModel
- Included verbose during training in BaseModel class

1.2.0
- Introduced Skip Connection :D
- Included callbacks parameter for .fit() function in BaseModel class

1.3.0
- Added data augmentation during loading dataset 
  - Currently only horizontal and vertical shifts
  - Potential augmentations:
    - Rotation
    - Adding noise